In [ ]:
#!pip install selenium

In [1]:
import pandas as pd
import csv
import time
import re

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, TimeoutException

In [3]:
df = pd.read_csv('Dataset/Postcode_LSOA_MSOA/London postcodes.csv')
df.head()

C:\Users\USER\AppData\Local\Temp\ipykernel_22544\2772030986.py:1: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Dataset/Postcode_LSOA_MSOA/London postcodes.csv')


,Postcode,In Use?,Latitude,Longitude,Easting,Northing,Grid Ref,County,District,Ward,...,ITL level 2,ITL level 3,UPRNs,Distance to sea,LSOA21 Code,Lower layer super output area 2021,MSOA21 Code,Middle layer super output area 2021,Census output area 2021,IMD decile
0,BR1 1AA,Yes,51.401546,0.015415,540291,168873,TQ402688,Greater London,Bromley,Bromley Town,...,Outer London - South,Bromley,"10070014435,10070014436,10070014437,1007001443...",28.0730,E01034386,Bromley 018G,E02000144,Bromley South,E00182600,8
1,BR1 1AB,Yes,51.406333,0.015208,540262,169405,TQ402694,Greater London,Bromley,Bromley Town,...,Outer London - South,Bromley,"10070008860,10070008861,10070008862,1007000886...",27.9776,E01000676,Bromley 008B,E02000134,Bromley North & Sundridge,E00003255,5
2,BR1 1AD,No,51.400057,0.016715,540386,168710,TQ403687,Greater London,Bromley,Bromley Town,...,Outer London - South,Bromley,NaN,28.0211,E01034386,Bromley 018G,E02000144,Bromley South,E00182639,8
3,BR1 1AE,Yes,51.404543,0.014195,540197,169204,TQ401692,Greater London,Bromley,Bromley Town,...,Outer London - South,Bromley,"10003640209,10070000614,10070002658,1007000265...",28.0861,E01000677,Bromley 018C,E02000144,Bromley South,E00003266,7
4,BR1 1AF,Yes,51.401392,0.014948,540259,168855,TQ402688,Greater London,Bromley,Bromley Town,...,Outer London - South,Bromley,"10070014484,10070014485,10070014486,1007001448...",28.1083,E01034386,Bromley 018G,E02000144,Bromley South,E00182600,8


In [4]:
sub_cols = ['Postcode', 'In Use?', 'Latitude', 'Longitude', 'District', 'District Code', 
                  'Introduced', 'Terminated', 'London zone', 'User Type', 'Nearest station', 
                  'Distance to station', 'Postcode area', 'Postcode district', 'IMD decile']

df_sub = df[sub_cols]
df_sub.head()

,Postcode,In Use?,Latitude,Longitude,District,District Code,Introduced,Terminated,London zone,User Type,Nearest station,Distance to station,Postcode area,Postcode district,IMD decile
0,BR1 1AA,Yes,51.401546,0.015415,Bromley,E09000006,2016-05-01,NaN,5,0,Bromley South,0.218254,BR,BR1,8
1,BR1 1AB,Yes,51.406333,0.015208,Bromley,E09000006,2012-03-01,NaN,4,0,Bromley North,0.253666,BR,BR1,5
2,BR1 1AD,No,51.400057,0.016715,Bromley,E09000006,2014-09-01,2017-09-01,5,1,Bromley South,0.044559,BR,BR1,8
3,BR1 1AE,Yes,51.404543,0.014195,Bromley,E09000006,2008-08-01,NaN,4,0,Bromley North,0.462939,BR,BR1,7
4,BR1 1AF,Yes,51.401392,0.014948,Bromley,E09000006,2015-05-01,NaN,5,0,Bromley South,0.227662,BR,BR1,8


In [5]:
df_fil = df_sub[df_sub['In Use?'] == 'Yes']
df_fil.head()

,Postcode,In Use?,Latitude,Longitude,District,District Code,Introduced,Terminated,London zone,User Type,Nearest station,Distance to station,Postcode area,Postcode district,IMD decile
0,BR1 1AA,Yes,51.401546,0.015415,Bromley,E09000006,2016-05-01,NaN,5,0,Bromley South,0.218254,BR,BR1,8
1,BR1 1AB,Yes,51.406333,0.015208,Bromley,E09000006,2012-03-01,NaN,4,0,Bromley North,0.253666,BR,BR1,5
3,BR1 1AE,Yes,51.404543,0.014195,Bromley,E09000006,2008-08-01,NaN,4,0,Bromley North,0.462939,BR,BR1,7
4,BR1 1AF,Yes,51.401392,0.014948,Bromley,E09000006,2015-05-01,NaN,5,0,Bromley South,0.227662,BR,BR1,8
5,BR1 1AG,Yes,51.401392,0.014948,Bromley,E09000006,2015-09-01,NaN,5,0,Bromley South,0.227662,BR,BR1,8


In [8]:
df_fil.info()

<class 'pandas.core.frame.DataFrame'>
Index: 180677 entries, 0 to 329571
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Postcode             180677 non-null  object 
 1   In Use?              180677 non-null  object 
 2   Latitude             180677 non-null  float64
 3   Longitude            180677 non-null  float64
 4   District             180677 non-null  object 
 5   District Code        180677 non-null  object 
 6   Introduced           180677 non-null  object 
 7   Terminated           0 non-null       object 
 8   London zone          180677 non-null  int64  
 9   User Type            180677 non-null  int64  
 10  Nearest station      180677 non-null  object 
 11  Distance to station  180677 non-null  float64
 12  Postcode area        180677 non-null  object 
 13  Postcode district    180677 non-null  object 
 14  IMD decile           180677 non-null  int64  
dtypes: float64(3), int64(3

In [7]:
# Ensure the Postcode field is of string type
df_fil['Postcode'] = df_fil['Postcode'].astype(str)

C:\Users\USER\AppData\Local\Temp\ipykernel_22544\863718835.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fil['Postcode'] = df_fil['Postcode'].astype(str)


In [10]:
# Extract the first four characters of each postcode
df['Postcode_prefix'] = df['Postcode'].apply(lambda x: x[:5])

# Calculate the count of different postcode prefixes
unique_postcodes = df['Postcode_prefix'].unique()
num_unique_postcodes = len(unique_postcodes)

print("Unique postcode prefixes count:", num_unique_postcodes)
print("Unique postcode prefixes:", unique_postcodes)

Unique postcode prefixes count: 3388
Unique postcode prefixes: ['BR1 1' 'BR1 2' 'BR1 3' ... 'WD3 2' 'WD3 8' 'WD6 2']


<br>
<br>

In [7]:
postcode_list_1 = df_fil['Postcode district'].unique()
print(postcode_list_1)

['BR1' 'BR2' 'BR3' 'BR4' 'BR5' 'BR6' 'BR7' 'BR8' 'CM13' 'CM14' 'CR0' 'CR2'
 'CR3' 'CR4' 'CR44' 'CR5' 'CR6' 'CR7' 'CR8' 'CR9' 'CR90' 'DA1' 'DA14'
 'DA15' 'DA16' 'DA17' 'DA18' 'DA5' 'DA6' 'DA7' 'DA8' 'E1' 'E10' 'E11'
 'E12' 'E13' 'E14' 'E15' 'E16' 'E17' 'E18' 'E1W' 'E2' 'E20' 'E3' 'E4' 'E5'
 'E6' 'E7' 'E8' 'E9' 'E98' 'EC1A' 'EC1M' 'EC1N' 'EC1P' 'EC1R' 'EC1V'
 'EC1Y' 'EC2A' 'EC2M' 'EC2N' 'EC2P' 'EC2R' 'EC2V' 'EC2Y' 'EC3A' 'EC3M'
 'EC3N' 'EC3P' 'EC3R' 'EC3V' 'EC4A' 'EC4M' 'EC4N' 'EC4P' 'EC4R' 'EC4V'
 'EC4Y' 'EN1' 'EN2' 'EN3' 'EN4' 'EN5' 'EN6' 'EN7' 'EN8' 'EN9' 'HA0' 'HA1'
 'HA2' 'HA3' 'HA4' 'HA5' 'HA6' 'HA7' 'HA8' 'HA9' 'IG1' 'IG11' 'IG2' 'IG3'
 'IG4' 'IG5' 'IG6' 'IG7' 'IG8' 'IG9' 'KT1' 'KT17' 'KT18' 'KT19' 'KT2'
 'KT22' 'KT3' 'KT4' 'KT5' 'KT6' 'KT7' 'KT8' 'KT9' 'N1' 'N10' 'N11' 'N12'
 'N13' 'N14' 'N15' 'N16' 'N17' 'N18' 'N19' 'N1C' 'N1P' 'N2' 'N20' 'N21'
 'N22' 'N3' 'N4' 'N5' 'N6' 'N7' 'N8' 'N81' 'N9' 'NW1' 'NW10' 'NW11' 'NW1W'
 'NW2' 'NW26' 'NW3' 'NW4' 'NW5' 'NW6' 'NW7' 'NW8' 'NW9' 'RM1'

In [8]:
camden_data = df_fil.loc[df_fil['District'] == 'Camden']

camden_data.head()

,Postcode,In Use?,Latitude,Longitude,District,District Code,Introduced,Terminated,London zone,User Type,Nearest station,Distance to station,Postcode area,Postcode district,IMD decile
58822,EC1M 3HA,Yes,51.519286,-0.105847,Camden,E09000007,1980-01-01,NaN,1,0,Farringdon,0.107355,EC,EC1M,5
58838,EC1M 3JB,Yes,51.520425,-0.106119,Camden,E09000007,1980-01-01,NaN,1,0,Farringdon,0.069483,EC,EC1M,5
58840,EC1M 3JF,Yes,51.519901,-0.105954,Camden,E09000007,2018-05-01,NaN,1,0,Farringdon,0.059571,EC,EC1M,5
58845,EC1M 3JP,Yes,51.520389,-0.106106,Camden,E09000007,1998-12-01,NaN,1,0,Farringdon,0.067085,EC,EC1M,5
58846,EC1M 3JQ,Yes,51.521005,-0.106510,Camden,E09000007,1998-12-01,NaN,1,0,Farringdon,0.129795,EC,EC1M,5


In [9]:
postcode_camden_list2 = camden_data['Postcode'].tolist()

print(len(postcode_camden_list2))
print(postcode_camden_list2)

8225
['EC1M 3HA', 'EC1M 3JB', 'EC1M 3JF', 'EC1M 3JP', 'EC1M 3JQ', 'EC1M 3JU', 'EC1M 3JY', 'EC1M 3LL', 'EC1M 3LN', 'EC1N 2AB', 'EC1N 2DY', 'EC1N 2NS', 'EC1N 2ST', 'EC1N 2SW', 'EC1N 2TD', 'EC1N 6AA', 'EC1N 6RA', 'EC1N 6RY', 'EC1N 6SA', 'EC1N 6SJ', 'EC1N 6SN', 'EC1N 6TD', 'EC1N 6TE', 'EC1N 7AA', 'EC1N 7AB', 'EC1N 7AD', 'EC1N 7AE', 'EC1N 7AQ', 'EC1N 7RA', 'EC1N 7RB', 'EC1N 7RD', 'EC1N 7RE', 'EC1N 7RF', 'EC1N 7RJ', 'EC1N 7RR', 'EC1N 7RS', 'EC1N 7RU', 'EC1N 7SB', 'EC1N 7SD', 'EC1N 7SE', 'EC1N 7SN', 'EC1N 7SP', 'EC1N 7SQ', 'EC1N 7SR', 'EC1N 7SS', 'EC1N 7ST', 'EC1N 7SU', 'EC1N 7SX', 'EC1N 7SY', 'EC1N 7TB', 'EC1N 7TE', 'EC1N 7TJ', 'EC1N 7TN', 'EC1N 7TP', 'EC1N 7TR', 'EC1N 7TS', 'EC1N 7TT', 'EC1N 7TX', 'EC1N 7TY', 'EC1N 7UA', 'EC1N 7UB', 'EC1N 7UD', 'EC1N 7UE', 'EC1N 7UH', 'EC1N 7UJ', 'EC1N 7UL', 'EC1N 7UN', 'EC1N 7UP', 'EC1N 7UQ', 'EC1N 7UR', 'EC1N 7UT', 'EC1N 7UU', 'EC1N 7UY', 'EC1N 8AA', 'EC1N 8AD', 'EC1N 8AF', 'EC1N 8AG', 'EC1N 8AH', 'EC1N 8AN', 'EC1N 8AS', 'EC1N 8AT', 'EC1N 8AW', 'EC1N 8BA'

In [10]:
postcode_camden_list2 = camden_data['Postcode'].tolist()

# Filter out postcodes starting with 'NW1W'
filtered_postcodes_camden = [pc for pc in postcode_camden_list2 if not pc.startswith('NW1W')]

# Check the number of postcodes in the filtered list and print the filtered list
print(len(filtered_postcodes_camden))
print(filtered_postcodes_camden)

7188
['EC1M 3HA', 'EC1M 3JB', 'EC1M 3JF', 'EC1M 3JP', 'EC1M 3JQ', 'EC1M 3JU', 'EC1M 3JY', 'EC1M 3LL', 'EC1M 3LN', 'EC1N 2AB', 'EC1N 2DY', 'EC1N 2NS', 'EC1N 2ST', 'EC1N 2SW', 'EC1N 2TD', 'EC1N 6AA', 'EC1N 6RA', 'EC1N 6RY', 'EC1N 6SA', 'EC1N 6SJ', 'EC1N 6SN', 'EC1N 6TD', 'EC1N 6TE', 'EC1N 7AA', 'EC1N 7AB', 'EC1N 7AD', 'EC1N 7AE', 'EC1N 7AQ', 'EC1N 7RA', 'EC1N 7RB', 'EC1N 7RD', 'EC1N 7RE', 'EC1N 7RF', 'EC1N 7RJ', 'EC1N 7RR', 'EC1N 7RS', 'EC1N 7RU', 'EC1N 7SB', 'EC1N 7SD', 'EC1N 7SE', 'EC1N 7SN', 'EC1N 7SP', 'EC1N 7SQ', 'EC1N 7SR', 'EC1N 7SS', 'EC1N 7ST', 'EC1N 7SU', 'EC1N 7SX', 'EC1N 7SY', 'EC1N 7TB', 'EC1N 7TE', 'EC1N 7TJ', 'EC1N 7TN', 'EC1N 7TP', 'EC1N 7TR', 'EC1N 7TS', 'EC1N 7TT', 'EC1N 7TX', 'EC1N 7TY', 'EC1N 7UA', 'EC1N 7UB', 'EC1N 7UD', 'EC1N 7UE', 'EC1N 7UH', 'EC1N 7UJ', 'EC1N 7UL', 'EC1N 7UN', 'EC1N 7UP', 'EC1N 7UQ', 'EC1N 7UR', 'EC1N 7UT', 'EC1N 7UU', 'EC1N 7UY', 'EC1N 8AA', 'EC1N 8AD', 'EC1N 8AF', 'EC1N 8AG', 'EC1N 8AH', 'EC1N 8AN', 'EC1N 8AS', 'EC1N 8AT', 'EC1N 8AW', 'EC1N 8BA'

In [25]:
test_list = ['SW10 0', 'SW10 0AA']

In [11]:
# Set up WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Prepare data storage
data_list = []

for postcode in filtered_postcodes_camden:
    # Visit the website
    driver.get("https://www.tax.service.gov.uk/check-council-tax-band/search")

    try:
        # Wait and enter the postal code
        postcode_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "postcode"))
        )
        postcode_input.clear()
        postcode_input.send_keys(postcode)  # Enter the specific postal code
        search_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "govuk-button"))
        )
        search_button.click()

        # Wait for the first page of results to fully load
        WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, ".govuk-table__cell a.govuk-link"))
        )

        while True:
            # Collect data from the current page
            links = driver.find_elements(By.CSS_SELECTOR, ".govuk-table__cell a.govuk-link")
            for link in links:
                title_text = link.get_attribute('title')  # Retrieve the 'title' attribute containing detailed information

                address_match = re.search(r"Property details for (.+) \(", title_text)
                band_match = re.search(r"\(Band (\w)\)", title_text)
                deleted_match = re.search(r"\(Deleted\)", title_text)

                if address_match:
                    full_address = address_match.group(1)
                    council_tax_band = 'Deleted' if deleted_match else band_match.group(1) if band_match else 'N/A'
                    address_parts = full_address.rsplit(',', 2)
                    address = ', '.join(address_parts[:-2])
                    postcode = address_parts[-1].strip()
                    # Finding the local authority using XPath
                    local_authority = link.find_element(By.XPATH, './ancestor::td/following-sibling::td/a').text

                    data_list.append([address, postcode, council_tax_band, local_authority])

            # Try to find and click the "Next" button
            try:
                next_button = driver.find_element(By.XPATH, "//span[contains(text(), 'Next')]/..")
                next_button.click()
                WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.CSS_SELECTOR, ".govuk-table__cell a.govuk-link"))
                )
                time.sleep(2)  # Pause for 2 seconds to allow the page to load
            except NoSuchElementException:
                print(f"Reached the last page for postcode {postcode}.")
                break
    except TimeoutException:
        print(f"No data available for postcode {postcode} or page load timeout.")

# Write the data to a CSV file
with open('council_tax_band_camden2.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Address", "Postcode", "Council Tax Band", "Local Authority"])
    writer.writerows(data_list)

# Close the browser
driver.quit()

Reached the last page for postcode EC1M 3HA.
Reached the last page for postcode EC1M 3JB.
No data available for postcode EC1M 3JF or page load timeout.
No data available for postcode EC1M 3JP or page load timeout.
Reached the last page for postcode EC1M 3JQ.
No data available for postcode EC1M 3JU or page load timeout.
No data available for postcode EC1M 3JY or page load timeout.
Reached the last page for postcode EC1M 3LL.
No data available for postcode EC1M 3LN or page load timeout.
No data available for postcode EC1N 2AB or page load timeout.
No data available for postcode EC1N 2DY or page load timeout.
Reached the last page for postcode EC1N 2NS.
No data available for postcode EC1N 2ST or page load timeout.
No data available for postcode EC1N 2SW or page load timeout.
No data available for postcode EC1N 2TD or page load timeout.
No data available for postcode EC1N 6AA or page load timeout.
No data available for postcode EC1N 6RA or page load timeout.
Reached the last page for postc

Reached the last page for postcode EC1R 4PQ.
Reached the last page for postcode EC1R 4PT.
No data available for postcode EC1R 4SR or page load timeout.
Reached the last page for postcode EC1R 4TD.
No data available for postcode EC1R 4TF or page load timeout.
No data available for postcode EC1R 5AA or page load timeout.
No data available for postcode EC1R 5AB or page load timeout.
No data available for postcode EC1R 5AR or page load timeout.
Reached the last page for postcode EC1R 5BD.
No data available for postcode EC1R 5BL or page load timeout.
No data available for postcode EC1R 5BU or page load timeout.
No data available for postcode EC1R 5BX or page load timeout.
Reached the last page for postcode EC1R 5BY.
Reached the last page for postcode EC1R 5DB.
No data available for postcode EC1R 5DD or page load timeout.
No data available for postcode EC1R 5DE or page load timeout.
No data available for postcode EC1R 5DF or page load timeout.
No data available for postcode EC1R 5DG or page 

Reached the last page for postcode N6 6AH.
Reached the last page for postcode N6 6AJ.
Reached the last page for postcode N6 6AL.
Reached the last page for postcode N6 6AP.
Reached the last page for postcode N6 6AR.
Reached the last page for postcode N6 6AS.
Reached the last page for postcode N6 6AT.
Reached the last page for postcode N6 6BA.
Reached the last page for postcode N6 6BB.
Reached the last page for postcode N6 6BD.
Reached the last page for postcode N6 6BE.
Reached the last page for postcode N6 6BH.
Reached the last page for postcode N6 6BJ.
Reached the last page for postcode N6 6BL.
Reached the last page for postcode N6 6BP.
Reached the last page for postcode N6 6BS.
Reached the last page for postcode N6 6BT.
Reached the last page for postcode N6 6BU.
Reached the last page for postcode N6 6DA.
Reached the last page for postcode N6 6DB.
Reached the last page for postcode N6 6DD.
Reached the last page for postcode N6 6DJ.
Reached the last page for postcode N6 6DP.
Reached the

Reached the last page for postcode NW1 0HX.
Reached the last page for postcode NW1 0HY.
Reached the last page for postcode NW1 0JA.
Reached the last page for postcode NW1 0JE.
Reached the last page for postcode NW1 0JF.
Reached the last page for postcode NW1 0JG.
Reached the last page for postcode NW1 0JH.
Reached the last page for postcode NW1 0JL.
Reached the last page for postcode NW1 0JN.
Reached the last page for postcode NW1 0JP.
Reached the last page for postcode NW1 0JR.
Reached the last page for postcode NW1 0JS.
Reached the last page for postcode NW1 0JT.
Reached the last page for postcode NW1 0JU.
Reached the last page for postcode NW1 0JW.
Reached the last page for postcode NW1 0JX.
Reached the last page for postcode NW1 0JY.
Reached the last page for postcode NW1 0JZ.
Reached the last page for postcode NW1 0LA.
Reached the last page for postcode NW1 0LB.
Reached the last page for postcode NW1 0LD.
Reached the last page for postcode NW1 0LE.
Reached the last page for postco

Reached the last page for postcode NW1 1LA.
Reached the last page for postcode NW1 1LB.
Reached the last page for postcode NW1 1LD.
Reached the last page for postcode NW1 1LE.
Reached the last page for postcode NW1 1LG.
Reached the last page for postcode NW1 1LH.
Reached the last page for postcode NW1 1LJ.
Reached the last page for postcode NW1 1LL.
Reached the last page for postcode NW1 1LN.
Reached the last page for postcode NW1 1LQ.
Reached the last page for postcode NW1 1LR.
Reached the last page for postcode NW1 1LS.
Reached the last page for postcode NW1 1LT.
No data available for postcode NW1 1LW or page load timeout.
Reached the last page for postcode NW1 1LX.
Reached the last page for postcode NW1 1LY.
Reached the last page for postcode NW1 1NA.
Reached the last page for postcode NW1 1NB.
Reached the last page for postcode NW1 1NH.
Reached the last page for postcode NW1 1NJ.
Reached the last page for postcode NW1 1NL.
Reached the last page for postcode NW1 1NN.
Reached the las

...

Reached the last page for postcode WC1N 3PA.
Reached the last page for postcode WC1N 3PB.
Reached the last page for postcode WC1N 3PD.
Reached the last page for postcode WC1N 3PE.
Reached the last page for postcode WC1N 3PF.
Reached the last page for postcode WC1N 3PG.
Reached the last page for postcode WC1N 3PH.
Reached the last page for postcode WC1N 3PJ.
Reached the last page for postcode WC1N 3PR.
Reached the last page for postcode WC1N 3PS.
Reached the last page for postcode WC1N 3PX.
Reached the last page for postcode WC1N 3PZ.
Reached the last page for postcode WC1N 3QA.
Reached the last page for postcode WC1N 3QH.
No data available for postcode WC1N 3QL or page load timeout.
No data available for postcode WC1N 3QN or page load timeout.
No data available for postcode WC1N 3QS or page load timeout.
Reached the last page for postcode WC1N 3QT.
No data available for postcode WC1N 3QU or page load timeout.
No data available for postcode WC1N 3QW or page load timeout.
Reached the las

Reached the last page for postcode WC1V 7AA.
No data available for postcode WC1V 7AB or page load timeout.
No data available for postcode WC1V 7AP or page load timeout.
No data available for postcode WC1V 7BD or page load timeout.
No data available for postcode WC1V 7BF or page load timeout.
No data available for postcode WC1V 7BH or page load timeout.
Reached the last page for postcode WC1V 7BW.
No data available for postcode WC1V 7DA or page load timeout.
Reached the last page for postcode WC1V 7DN.
Reached the last page for postcode WC1V 7DX.
No data available for postcode WC1V 7DZ or page load timeout.
Reached the last page for postcode WC1V 7EE.
No data available for postcode WC1V 7EG or page load timeout.
No data available for postcode WC1V 7EJ or page load timeout.
No data available for postcode WC1V 7EN or page load timeout.
Reached the last page for postcode WC1V 7EP.
No data available for postcode WC1V 7EW or page load timeout.
No data available for postcode WC1V 7EX or page 

Reached the last page for postcode WC1X 8QF.
Reached the last page for postcode WC1X 8QH.
No data available for postcode WC1X 8QJ or page load timeout.
Reached the last page for postcode WC1X 8QL.
Reached the last page for postcode WC1X 8QP.
No data available for postcode WC1X 8QR or page load timeout.
No data available for postcode WC1X 8QS or page load timeout.
Reached the last page for postcode WC1X 8QT.
Reached the last page for postcode WC1X 8QY.
No data available for postcode WC1X 8QZ or page load timeout.
No data available for postcode WC1X 8RA or page load timeout.
Reached the last page for postcode WC1X 8RB.
Reached the last page for postcode WC1X 8RH.
No data available for postcode WC1X 8RX or page load timeout.
Reached the last page for postcode WC1X 8SF.
No data available for postcode WC1X 8SH or page load timeout.
Reached the last page for postcode WC1X 8SL.
Reached the last page for postcode WC1X 8SP.
No data available for postcode WC1X 8TA or page load timeout.
Reached t

No data available for postcode WC2A 3XL or page load timeout.
Reached the last page for postcode WC2A 3XT.
No data available for postcode WC2B 4AU or page load timeout.
No data available for postcode WC2B 4BA or page load timeout.
Reached the last page for postcode WC2B 5AA.
No data available for postcode WC2B 5AB or page load timeout.
No data available for postcode WC2B 5AD or page load timeout.
No data available for postcode WC2B 5AE or page load timeout.
No data available for postcode WC2B 5AF or page load timeout.
No data available for postcode WC2B 5AH or page load timeout.
Reached the last page for postcode WC2B 5AN.
Reached the last page for postcode WC2B 5AP.
No data available for postcode WC2B 5AY or page load timeout.
No data available for postcode WC2B 5AZ or page load timeout.
Reached the last page for postcode WC2B 5BB.
No data available for postcode WC2B 5BD or page load timeout.
No data available for postcode WC2B 5BH or page load timeout.
Reached the last page for postc

No data available for postcode WC2H 8LP or page load timeout.
Reached the last page for postcode WC2H 8LQ.
Reached the last page for postcode WC2H 8LS.
Reached the last page for postcode WC2H 8LW.
No data available for postcode WC2H 8LX or page load timeout.
Reached the last page for postcode WC2H 8LZ.
Reached the last page for postcode WC2H 8NE.
No data available for postcode WC2H 8NH or page load timeout.
Reached the last page for postcode WC2H 8NJ.
No data available for postcode WC2H 8NU or page load timeout.
No data available for postcode WC2H 8PW or page load timeout.
No data available for postcode WC2H 9AA or page load timeout.
Reached the last page for postcode WC2H 9AB.
No data available for postcode WC2H 9AG or page load timeout.
No data available for postcode WC2H 9AH or page load timeout.
Reached the last page for postcode WC2H 9AJ.
No data available for postcode WC2H 9AN or page load timeout.
Reached the last page for postcode WC2H 9AP.
Reached the last page for postcode WC

In [15]:
kensington_chelsea_data = df_fil.loc[df_fil['District'] == 'Kensington and Chelsea']

kensington_chelsea_data.head()

,Postcode,In Use?,Latitude,Longitude,District,District Code,Introduced,Terminated,London zone,User Type,Nearest station,Distance to station,Postcode area,Postcode district,IMD decile
218237,SW10 0AB,Yes,51.478206,-0.183053,Kensington and Chelsea,E09000020,2010-08-01,NaN,2,0,Imperial Wharf,0.362414,SW,SW10,4
218238,SW10 0AD,Yes,51.485945,-0.180470,Kensington and Chelsea,E09000020,1980-01-01,NaN,1,0,Gloucester Road,0.974046,SW,SW10,7
218239,SW10 0AE,Yes,51.485776,-0.180621,Kensington and Chelsea,E09000020,1992-08-01,NaN,1,0,Gloucester Road,0.990195,SW,SW10,7
218240,SW10 0AF,Yes,51.481480,-0.183053,Kensington and Chelsea,E09000020,2014-05-01,NaN,2,0,Imperial Wharf,0.726045,SW,SW10,5
218241,SW10 0AG,Yes,51.485807,-0.180317,Kensington and Chelsea,E09000020,1980-01-01,NaN,1,0,Gloucester Road,0.991347,SW,SW10,7


In [22]:
postcode_test = ['SW10 0']

#### Adjust the method of scraping the Local Authority data (using XPath)

In [24]:
# Set up WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# Prepare data storage
data_list = []

for postcode in postcode_test:
    # Visit the website
    driver.get("https://www.tax.service.gov.uk/check-council-tax-band/search")

    try:
        # Wait and enter the postal code
        postcode_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "postcode"))
        )
        postcode_input.clear()
        postcode_input.send_keys(postcode)  # Enter the specific postal code
        search_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "govuk-button"))
        )
        search_button.click()

        # Wait for the first page of results to fully load
        WebDriverWait(driver, 10).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, ".govuk-table__cell a.govuk-link"))
        )

        while True:
            # Collect data from the current page
            links = driver.find_elements(By.CSS_SELECTOR, ".govuk-table__cell a.govuk-link")
            for link in links:
                title_text = link.get_attribute('title')  # Retrieve the 'title' attribute containing detailed information

                address_match = re.search(r"Property details for (.+) \(", title_text)
                band_match = re.search(r"\(Band (\w)\)", title_text)
                deleted_match = re.search(r"\(Deleted\)", title_text)

                if address_match:
                    full_address = address_match.group(1)
                    council_tax_band = 'Deleted' if deleted_match else band_match.group(1) if band_match else 'N/A'
                    address_parts = full_address.rsplit(',', 2)
                    address = ', '.join(address_parts[:-2])
                    postcode = address_parts[-1].strip()
                    
                    # Finding the local authority using XPath
                    local_authority = link.find_element(By.XPATH, './ancestor::td/following-sibling::td/a').text

                    data_list.append([address, postcode, council_tax_band, local_authority])

            # Try to find and click the "Next" button
            try:
                next_button = driver.find_element(By.XPATH, "//span[contains(text(), 'Next')]/..")
                next_button.click()
                WebDriverWait(driver, 10).until(
                    EC.visibility_of_element_located((By.CSS_SELECTOR, ".govuk-table__cell a.govuk-link"))
                )
                time.sleep(2)  # Pause for 2 seconds to allow the page to load
            except NoSuchElementException:
                print(f"Reached the last page for postcode {postcode}.")
                break
    except TimeoutException:
        print(f"No data available for postcode {postcode} or page load timeout.")

# Write the data to a CSV file
with open('council_tax_band_camden_test.csv', 'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(["Address", "Postcode", "Council Tax Band", "Local Authority"])
    writer.writerows(data_list)

# Close the browser
driver.quit()

Reached the last page for postcode SW10 0XB.


In [16]:
postcode_kc_list = kensington_chelsea_data['Postcode'].tolist()

print(postcode_kc_list)

['SW10 0AB', 'SW10 0AD', 'SW10 0AE', 'SW10 0AF', 'SW10 0AG', 'SW10 0AH', 'SW10 0AJ', 'SW10 0AL', 'SW10 0AN', 'SW10 0AP', 'SW10 0AQ', 'SW10 0AR', 'SW10 0AS', 'SW10 0AU', 'SW10 0AW', 'SW10 0AX', 'SW10 0AY', 'SW10 0AZ', 'SW10 0BB', 'SW10 0BG', 'SW10 0BH', 'SW10 0BJ', 'SW10 0BL', 'SW10 0BN', 'SW10 0BP', 'SW10 0BQ', 'SW10 0BS', 'SW10 0BT', 'SW10 0BU', 'SW10 0BW', 'SW10 0BX', 'SW10 0BY', 'SW10 0BZ', 'SW10 0DA', 'SW10 0DB', 'SW10 0DE', 'SW10 0DF', 'SW10 0DG', 'SW10 0DJ', 'SW10 0DL', 'SW10 0DN', 'SW10 0DQ', 'SW10 0DR', 'SW10 0DS', 'SW10 0DT', 'SW10 0DW', 'SW10 0DX', 'SW10 0DY', 'SW10 0DZ', 'SW10 0EA', 'SW10 0EB', 'SW10 0ED', 'SW10 0EE', 'SW10 0EF', 'SW10 0EG', 'SW10 0EH', 'SW10 0EJ', 'SW10 0EL', 'SW10 0EN', 'SW10 0EP', 'SW10 0EQ', 'SW10 0ER', 'SW10 0ES', 'SW10 0ET', 'SW10 0EW', 'SW10 0EX', 'SW10 0EY', 'SW10 0EZ', 'SW10 0FU', 'SW10 0HA', 'SW10 0HD', 'SW10 0HE', 'SW10 0HF', 'SW10 0HH', 'SW10 0HJ', 'SW10 0HL', 'SW10 0HN', 'SW10 0HP', 'SW10 0HR', 'SW10 0HS', 'SW10 0HT', 'SW10 0HU', 'SW10 0HX', 'SW